# Data Comparasion: Mail Number Between ACC & TT

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from multiprocessing import Pool
import gc

# ACC
這個部分主要是提取ACC中的掛號號碼起訖號(ACC35, ACC36)，並將兩者合併成一個ACC的mail list(裡頭包含ACC所有的掛號信件號碼)。

In [2]:
column_names = ['ACC01', 'ACC02', 'ACC03', 'ACC04', 'ACC05', 'ACC06', 'ACC07', 'ACC10', 
               'ACC11', 'ACC12', 'ACC13', 'ACC14', 'ACC15', 'ACC16', 'ACC17', 'ACC18', 'ACC19', 'ACC20', 
               'ACC21', 'ACC22', 'ACC23', 'ACC24', 'ACC25', 'ACC26', 'ACC27', 'ACC28', 'ACC29', 'ACC30', 
               'ACC31', 'ACC32', 'ACC33', 'ACC34', 'ACC35', 'ACC36', 'ACC37', 'ACC38', 'ACC39', 'ACC40', ]
df_acc_chunk = pd.read_csv("ACC.csv", header = None, names = column_names, chunksize = 1000000)

In [3]:
acc_mails_list = []

for chunk in df_acc_chunk:
#     acc_mails = ACC_chunk_preprocessing(chunk)
    acc_mails = chunk.loc[:, ['ACC35', 'ACC36']]
    acc_mails_list.append(acc_mails)
    
acc_mails_list = pd.concat(acc_mails_list)
gc.collect()

C:\Users\USER\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\USER\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (12,27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


84

In [4]:
acc_mails_list = acc_mails_list['ACC35'].append(acc_mails_list['ACC36'])
acc_mails_list = pd.concat(acc_mails_list)
acc_mails_list = acc_mails_list.unique()
gc.collect()

7

In [5]:
len(acc_mails_list)

29172925

# TT
此部分為提取TT中的信件掛號號碼，並進行一些處理，方便後續對ACC的掛號號碼進行比對。

In [6]:
colName = ['statusNum', 'mailNum', 'date', 'time', 'PostOfficeNum', 'others']
df_tt_chunk = pd.read_csv('TTS1.csv', header = None, names = colName, chunksize=10000000)

In [7]:
chunk_list = []

for chunk in df_tt_chunk:
    data = chunk
    break
data.head()

C:\Users\USER\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,statusNum,mailNum,date,time,PostOfficeNum,others
0,Y4,00000000000000,2018-01-01,09:49:04,330031,003
1,Y4,00000000000000,2018-01-01,09:58:08,330031,002
2,I4,00000000000000,2018-01-01,14:11:51,330031,
3,Y4,00000000000000,2018-01-01,14:23:32,330031,006
4,I4,00000000000000,2018-01-01,14:52:09,330031,


In [8]:
# data.dtypes

In [9]:
# data['mailNum'] = data['mailNum'].astype('str')

In [10]:
# mails = data.groupby('mailNum').agg(['count']).index.tolist()
# mails

In [11]:
# for i in range(0, len(mails)):
#     mails[i] = mails[i].replace(" ", "")
# #     print(item)
    
# mails[0:10]

In [12]:
def TT_chunk_preprocessing(chunk):    
    chunk['mailNum'] = chunk['mailNum'].astype('str')
    mails = chunk.groupby('mailNum').agg(['count']).index.tolist()
    gc.collect()
    return mails

In [13]:
tt_mails_list = []

for chunk in df_tt_chunk:
    mails = TT_chunk_preprocessing(chunk)
    tt_mails_list = tt_mails_list + mails
    
tt_mails_list = list(set(tt_mails_list))
for i in range(0, len(tt_mails_list)):
    tt_mails_list[i] = tt_mails_list[i].replace(" ", "")
gc.collect()

C:\Users\USER\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (1,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


0

In [14]:
len(tt_mails_list)

87292842

In [15]:
tt_mails_list[:10]

['01427790201516',
 '12243240507216',
 '23249810602718',
 '94177540005510407098',
 '55639310045130710005',
 '52878010045117',
 '91371310011210100005',
 '91719810006700104002',
 '05697780202417',
 '01103940506716']

# Compare
TT vs. ACC

將acc_mails_list和tt_mails_list進行比對，看acc_mails_list中的掛號號碼是否皆存在於tt_mails_list中。

In [18]:
# counterpart = acc_mails_list[acc_mails_list.isin(tt_mails_list)]
# len(counterpart)

In [19]:
result =  all(elem in tt_mails_list for elem in acc_mails_list)
result

False

結果發現大部分ACC中的掛號號碼存在tt_mails_list中，而有部分則沒有，除了為空值得資料之外，還需要進一步釐清原因。

# TODO

現在已經確認ACC和TT中的掛號號碼可以對應起來，下一步有幾件事情可以做：
* 可以將兩者資料合併，雖然這樣會比較容易處理資料，但資料量會變大許多，而且也會產生許多新的空值。
* 拆解掛號號碼，進一步提取掛號號碼所包含的內容。
* 將掛號號碼所提取出的資訊跟其他欄位資料進行比對。